In [8]:
# build poetry's book latex source code from .md file repository

In [9]:
# global definitions

MODE="he"

REPO_PATH = "../.."

DEBUG_ADD_PATH = False
CHECK_STARTS_WITH_UPPER = False

TEX_OUTPUT_FILENAME = "/tmp/output.tex"
PDF_OUTPUT_FOLDER = "/tmp"


if MODE == "en":
    POEM_PATH_SUBSTRING = "/songs/en/"
    FONT="Linden Hill"
    IGNORE_TITLES = ["Flower & Produce", "Stop Resenting"]
    TEX_ADD = r"""
"""
    
elif MODE == "he":
    POEM_PATH_SUBSTRING = "/songs/he/"
    FONT="Arial"
    IGNORE_TITLES = []
    TEX_ADD = r"""
\usepackage[bidi=basic,provide=*,english,hebrew]{babel}
\babelfont{rm}{FreeSerif}
"""
    
    


TEX_BLANK = r"""
\begin{titlepage}
  \centering
  \vfill
\end{titlepage}
"""

TEX_PREAMBLE = r"""
% compile with lualatex
\documentclass[12pt]{book}
\usepackage{calc,emptypage,fontspec,graphicx,microtype,verse,xcolor}

xxxADDxxx

% a free version of Goudy’s Deepdene from www.theleagueofmoveabletype.com:
\setmainfont{xxxFONTxxx}
\linespread{1.10344}

\setlength{\topskip}{\baselineskip}
\usepackage[a5paper,hcentering,heightrounded,includeheadfoot]{geometry}

\definecolor{myBGcolor}{HTML}{FFFFFF}
\definecolor{myTextcolor}{HTML}{000000}
\pagecolor{myBGcolor}
\color{myTextcolor}

% I’ve reduced the default left margin for verse to prevent lines from
% breaking:
\setlength{\leftmargini}{1em}

% One of many customizations made possible by the verse package:
\renewcommand{\poemtitlefont}{\normalfont\large\centering}

% This is said to help Kindles, but I don’t have one for testing:
\usepackage[a5,noinfo,center,frame,color=myBGcolor!99]{crop}
\crop[frame]

% toc - if you make one and if your font has anything resembling a bold
% weight, LaTeX will try to use it in your TOC.  Bold and poetry don’t
% mix; here’s a quick, brute-force method to get rid of all bold type:
\let\bfseries\mdseries

% Get rid of blank pages if this is to be read on screen:
%\let\cleardoublepage\clearpage

\usepackage{bookmark}
\hypersetup{hidelinks,pdftitle={The Lyrics},pdfauthor={Dror Kessler}}
\pagestyle{plain}
\begin{document}
\frontmatter

\begin{titlepage}
  \centering
  \vspace*{.1\textheight}

  \resizebox{.9\textwidth}{!}{\addfontfeatures{Kerning=Uppercase}The Lyrics}

  \vspace*{.1\textheight}

  {\huge Dror Kessler\par}

  \vspace*{.1\textheight}

  {xxxTIMESTAMPxxx\par}

\vfill

%  \includegraphics[scale=.36]{drorkessler}
\end{titlepage}

% blank page before contents
\begin{titlepage}
  \centering
  \vfill
\end{titlepage}

\tableofcontents

\mainmatter
"""

In [10]:
# produce list of .md files of individual poems
import os

def poem_files():
    result = []
    for path, subdirs, files in os.walk(REPO_PATH):
        for name in files:
            fullpath = os.path.join(path, name)
            if POEM_PATH_SUBSTRING in fullpath:
                result.append(fullpath)
                
    return result

#poem_files()

In [11]:
# parse a poem file
import string

def parse_poem_file(filename):
    poem = {"title": "", "blocks":[], "path":filename, "instrumental":False}
    block = []
    
    with open(filename, 'r', encoding='UTF-8') as file:
        while line := file.readline():
            line = line.strip();
            
            if line.startswith("#"):
                if line.endswith('\\'):
                    # titles should not end with backslash
                    print("WARN: title-with-backslash: %s: %s" % (filename, line))
                    line = line[:-1].strip()
                poem["title"] = line[2:]
            elif line.startswith("---"):
                break
            else:
                if line.endswith('\\'):
                    line = line[:-1].strip()
                if len(line):                    
                    # lines should not end with comma
                    if line.endswith(","):
                        print("WARN: ends-with-punct: %s: %s" % (filename, line))
                        
                    # remove fancy quotes
                    line = line.replace('“', '"')
                    line = line.replace('”', '"')
                    
                    # lines should start with a capital letter
                    if line[0] != '(':
                        line_np = line.translate(str.maketrans('', '', string.punctuation))
                        line_np = line_np.translate(str.maketrans('', '', '"'))
                        line_np = line_np.translate(str.maketrans('', '', "‘"))
                        if CHECK_STARTS_WITH_UPPER and not line_np[0].isupper():
                            print("WARN: line-start-not-upper: %s: %s" % (filename, line))
                        illegal = '“”'
                        for ch in line:
                            if ch in illegal:
                                print("WARN: lillegal char: %s: %s (%s)" % (filename, line, ch))
                    block.append(line)
                    if "(instrumental)" in line.lower():
                        poem["instrumental"] = True
                else:
                    if len(block):
                        poem["blocks"].append(block)
                    block = []

                
    if len(block):
        poem["blocks"].append(block)

    return poem
            
#parse_poem_file(poem_files()[0])

In [12]:
# build tex for a single poem

def poem_tex(poem):
    tex = []
    tex.append("\\poemtitle{" + poem["title"] + "}")
    
    tex.append("\\begin{verse}")

    if DEBUG_ADD_PATH:
        tex.append(poem["path"])
        tex.append("")

    for block in poem["blocks"]:
        for line in block[:-1]:
            tex.append(line + "\\\\*")
        tex.append(block[-1])
        tex.append("")
    
    tex.append("\\end{verse}")
    
    return "\n".join(tex).replace("&", "\\&")

#print(poem_tex(parse_poem_file(poem_files()[0])))

In [13]:
# create output tex
from datetime import date
from lorem_text import lorem
today = date.today()

with open(TEX_OUTPUT_FILENAME, 'w', encoding='UTF-8') as file:
    
    # write preamble
    preamble = TEX_PREAMBLE.replace("xxxTIMESTAMPxxx", today.strftime("%B, %Y"))
    preamble = preamble.replace("xxxFONTxxx", FONT)    
    preamble = preamble.replace("xxxADDxxx", TEX_ADD)    
    file.write(preamble)
    
    # get poems
    poems = [parse_poem_file(f) for f in poem_files()]
    
    # sort alphabetically
    poems = sorted(poems, key=lambda p: p["title"])
    print("%s poems" % len(poems))
    
    # establish range
    #hunt_inst = [0, 0, 0] succ
    hunt_inst = []
    p_from = 0
    p_to = len(poems)
    for inst in hunt_inst:
        p_size = (p_to - p_from) / 2
        if not inst:
            p_to -= p_size
        else:
            p_from += p_size
    p_from = int(round(p_from))
    p_to = int(round(p_to))
    
    #p_to -= 1
    
    # write poems
    print("writing poems [%d-%d)\n" % (p_from, p_to))
    for poem in poems[p_from:p_to]:
        if not poem["instrumental"] and not poem["title"] in IGNORE_TITLES:
            #print("%d: %s" % (p_from, poem["title"]))
            p_from += 1
            
            tex = poem_tex(poem)
            file.write(tex)
            file.write("\n")
            file.write("\\clearpage\n")
            file.write("\n")
        
    # add blank pages
    blanks = 0
    print("adding %d blanks" % blanks)
    for _ in range(blanks):
        poem = {"title": "X", "blocks":[], "path":"", "instrumental":False}
        for _ in range(2):
            block = []
            for _ in range(1):
                block.append(lorem.sentence())
            poem["blocks"].append(block)
        tex = poem_tex(poem)
        file.write(tex)
        file.write("\n")
        file.write("\\clearpage\n")
        file.write("\n")
        
    # close file
    file.write("\\end{document}\n");


120 poems
writing poems [0-120)

adding 0 blanks


In [14]:
# generate pdf
import os

# run twice to make sure toc index is correct
cmd = "lualatex --output-directory " + PDF_OUTPUT_FOLDER + " " + TEX_OUTPUT_FILENAME
for _ in range(2):
    os.system(cmd)
